In [ ]:
#| default_exp inout

# Input/Output
> Files reader and writer

In [ ]:
#| export
import tomli_w
import tomli

In [ ]:
#| export
def write_toml(fname, cfg):
    print(f'Creating {fname}')
    with open(fname, "wb") as f:
        tomli_w.dump(cfg, f)

In [ ]:
#| export
def read_toml(fname):
    with open(fname, "rb") as f:
        config = tomli.load(f)
    return config